In [1]:
import pandas as pd
import torch
from datasets import DatasetDict, load_from_disk

In [2]:
train_dataset = load_from_disk('/opt/ml/input/data/train_dataset')
test_dataset = load_from_disk('/opt/ml/input/data/test_dataset')

In [10]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [11]:
test_dataset

DatasetDict({
    validation: Dataset({
        features: ['question', 'id'],
        num_rows: 600
    })
})

In [12]:
train_df = pd.DataFrame(train_dataset['train'])
val_df = pd.DataFrame(train_dataset['validation'])
test_df = pd.DataFrame(test_dataset['validation'])

In [38]:
test_df

,question,id,cont_len
0,유령'은 어느 행성에서 지구로 왔는가?,mrc-1-000653,862
1,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,mrc-1-001113,621
2,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,mrc-0-002191,850
3,"마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?",mrc-0-003951,727
4,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,mrc-1-001272,609
...,...,...,...
595,타입 2 가이아 메모리을 만든 집단은?,mrc-0-002989,1321
596,장면이 정치보복에 반대하는 입장에서 처벌을 원치 않은 대상은?,mrc-0-001804,776
597,"콜드게임 중 어떠한 계기로 인해 잠시 중단된 뒤, 익일에 게임이 진행되는 것은?",mrc-0-003411,740
598,제2캐나다기갑여단이 상륙한 곳은?,mrc-0-003436,545


In [39]:
train_df['cont_len'] = train_df['context'].apply(lambda x : len(x) )
val_df['cont_len'] = train_df['context'].apply(lambda x : len(x) )

In [51]:
print(f'there are raw max sequnece lens, not tokenized')
print(f'train : {max(train_df["cont_len"])}')
print(f'val : {max(val_df["cont_len"])}')
# print(f'there are raw max sequnece len, not tokenized')

there are raw max sequnece lens, not tokenized
train : 2059
val : 1978


In [23]:
from transformers import AutoTokenizer
# 어떤 토크나이저에 따라 길이는 달라집니다. 항상 
tokenizer1 = AutoTokenizer.from_pretrained('klue/bert-base')
tokenizer2 = AutoTokenizer.from_pretrained('monologg/kobigbird-bert-base')

In [41]:
tokenizer1

PreTrainedTokenizerFast(name_or_path='klue/bert-base', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [45]:
tokenizer2.decode(tokenizer2.encode(train_df['context'][0]))

'[CLS] 미국 상의원 또는 미국 상원 ( United States Senate ) 은 양원제인 미국 의회의 상원이다. \\ n \\ n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1 / 3씩 상원의원을 새로 선출하여 연방에 보낸다. \\ n \\ n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할 ( 하원의 법안을 거부할 권한 등 ) 을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션 ( 공공건강보험기관 ) 의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다. 날짜 = 2017 - 02 - 05 [SEP]'

In [50]:
len(tokenizer2(train_df['context'][0])['input_ids'])

438

In [42]:
train_max = 0
for i,df in train_df.iterrows():
    tmp = len(tokenizer2(df['context'])['input_ids'])
    train_max = max(train_max,tmp)
train_max

1156

In [36]:
val_max = 0
for i,df in val_df.iterrows():
    tmp = len(tokenizer2(df['context'])['input_ids'])
    val_max = max(val_max,tmp)
val_max

1154